In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from my_functions import *
import seaborn as sns
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u
import glob

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))

In [ ]:
## Load QSO catalog
filename = ('/home/alberto/cosmos/JPAS_mocks_sep2021/'
    'JPAS_mocks_classification_01sep_model11/Fluxes/Qso_jpas_mock_flam_train.cat')

my_filter_order = np.arange(60)
my_filter_order[[-4, -3, -2, -1]] = np.array([1, 12, 28, 43])
my_filter_order[1:-4] += 1
my_filter_order[12:-4] += 1
my_filter_order[28:-4] += 1
my_filter_order[43:-4] += 1

qso_flx = pd.read_csv(
    filename, sep=' ', usecols=range(2, 2 + 60)
).to_numpy().T[my_filter_order]
qso_err = pd.read_csv(
    filename, sep=' ', usecols=range(2 + 60, 2 + 60 + 60)
).to_numpy().T[my_filter_order]
qso_zspec = pd.read_csv(filename, sep=' ', usecols=[127]).to_numpy().reshape(-1, )

# Randomly sample sources corresponding to 200 deg2
# idx = np.random.randint(0, 100000, 510 * 200)
idx = np.arange(100_000)
qso_flx = qso_flx[:, idx]
qso_err = qso_err[:, idx]
qso_zspec = qso_zspec[idx]

Lya_fts = pd.read_csv('csv/Lya_fts.csv')
EW_qso = np.abs(Lya_fts.LyaEW)[idx] / (qso_zspec + 1)

# Apply errors
np.random.seed(22)
# qso_flx += qso_err * np.random.normal(size=qso_err.shape)

In [ ]:
pm_flx = qso_flx
pm_err = qso_err
zspec = qso_zspec
EW_lya = EW_qso

N_qso = qso_flx.shape[1]

qso_dL = cosmo.luminosity_distance(qso_zspec).to(u.cm).value


qso_flambda = Lya_fts.LyaF * 1e-17

qso_L = np.log10(qso_flambda * 4*np.pi * qso_dL ** 2)

L_lya = qso_L
fline = qso_flambda

In [ ]:
w_lya = 1215.67 # A
N_sources = pm_flx.shape[1]
N_sources

In [ ]:
fig = plt.figure(figsize=(12, 6))

n = 84000
ax = plot_JPAS_source(pm_flx[:, n], pm_err[:, n])

In [ ]:
mag = flux_to_mag(pm_flx, w_central.reshape(-1, 1))
mag[np.isnan(mag)] = 99.

In [ ]:
# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(pm_flx, pm_err, IGM_T_correct=True)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(pm_flx, pm_err, IGM_T_correct=False)

In [ ]:
ew0lya_min = 0
ew0lya_max = 70
ew0lya_step = 8
ew0oth_min = 0
ew0oth_max = 400
ew0oth_step = 11

other_select_list = []
for ew0min in np.linspace(ew0oth_min, ew0oth_max, ew0oth_step):
    print(ew0min)
    line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other, ew0min,
        obs=True)
    other_lines = identify_lines(line_other, pm_flx, pm_err, first=False)

    other_select_list.append(other_lines)

lya_select_list = []
lya_z_nb = []
for ew0min in np.linspace(ew0lya_min, ew0lya_max, ew0lya_step):
    print(ew0min)
    line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, ew0min)
    lya_lines, lya_cont_lines = identify_lines(line, pm_flx, pm_err, first=True)
    z_nb_Arr = np.ones(N_sources) * -1 
    z_nb_Arr[np.where(np.array(lya_lines) != -1)] =\
        z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])
    
    lya_select_list.append(lya_lines)
    lya_z_nb.append(z_nb_Arr)

In [ ]:
mag_cut = 24
z_cut_min = 2.45
z_cut_max = 3.54

select_grid = np.zeros((ew0lya_step, ew0oth_step))
rightz_grid = np.zeros((ew0lya_step, ew0oth_step))

target_z = (zspec > z_cut_min) & (zspec < z_cut_max)
target = (
    (mag[-2] < mag_cut) & target_z & (EW_lya > 20)
)

for i in range(ew0lya_step):
    print(i)
    for j in range(ew0oth_step):
        nice_lya = (
            nice_lya_select(
                lya_select_list[i], other_select_list[j], pm_flx, pm_err,
                cont_est_other, lya_z_nb[i]
            )
        )
        nice_z = np.abs(lya_z_nb[i] - zspec) < 0.12
        
        z_nb_cut = (lya_z_nb[i] > z_cut_min) & (lya_z_nb[i] < z_cut_max)

        selected = (mag[-2] < mag_cut) & nice_lya & z_nb_cut
        select_grid[i, j] = len(np.where(selected)[0])
        rightz_grid[i, j] = len(np.where(selected & nice_z)[0])

In [ ]:
plt.hist(EW_lya[target] , bins=np.linspace(-100, 200, 50))
plt.show()

In [ ]:
N_target = len(np.where(target)[0])
purity = rightz_grid / select_grid
completeness = rightz_grid / N_target

In [ ]:
fig = plt.figure(figsize=(8, 8))

width = 0.5
height = 0.5
spacing = 0.06
cbar_width = 0.05

# Define axes
ax00 = fig.add_axes([0, height + 1.5 * spacing, width, height])
ax01 = fig.add_axes([width + spacing, height + 1.5 * spacing, width, height], sharey=ax00)
ax10 = fig.add_axes([0, 0, width, height], sharex=ax00)
ax11 = fig.add_axes([width + spacing, 0, width, height], sharex=ax01, sharey=ax10)
axcbar0 = fig.add_axes([2 * width + 1.5 * spacing, height + 1.5 * spacing, cbar_width, height])
axcbar1 = fig.add_axes([2 * width + 1.5 * spacing, 0, cbar_width, height])

# Plot stuff in the rectangles
vmax = np.max([np.max(rightz_grid), np.max(select_grid)])
cmap = 'Spectral'

sns.heatmap(rightz_grid, ax=ax00, vmin=0, vmax=vmax, cbar_ax=axcbar0, cmap=cmap)
sns.heatmap(select_grid, ax=ax01, vmin=0, vmax=vmax, cbar_ax=axcbar0, cmap=cmap)

sns.heatmap(purity, ax=ax10, vmin=0, vmax=1, cbar_ax=axcbar1, cmap=cmap)
sns.heatmap(completeness, ax=ax11, vmin=0, vmax=1, cbar=False, cmap=cmap)

ax00.invert_yaxis()
ax10.invert_yaxis()

# Axes ticks
xticks = range(ew0oth_step)
yticks = range(ew0lya_step)
xtick_labels = ['{0:0.0f}'.format(n) for n in np.linspace(ew0oth_min, ew0oth_max, ew0oth_step)]
ytick_labels = ['{0:0.0f}'.format(n) for n in np.linspace(ew0lya_min, ew0lya_max, ew0lya_step)]

ax00.set_yticks(yticks)
ax00.set_yticklabels(ytick_labels)
ax00.set_xticks(xticks)
ax00.set_xticklabels(xtick_labels, rotation='vertical')

ax10.set_yticks(yticks)
ax10.set_yticklabels(ytick_labels)
ax10.set_xticks(xticks)
ax10.set_xticklabels(xtick_labels, rotation='vertical')

ax11.set_xticks(xticks)
ax11.set_xticklabels(xtick_labels, rotation='vertical')
ax11.set_yticks(yticks)
ax11.set_yticklabels(ytick_labels)

ax01.set_xticks(xticks)
ax01.set_xticklabels(xtick_labels, rotation='vertical')
ax01.set_yticks(yticks)
ax01.set_yticklabels(ytick_labels)

# Axes labels
ylabel = r'Ly$\alpha$ EW$_0$ ($\AA$)'
xlabel = 'Other lines EW$_\mathrm{obs}$ ($\AA$)'
ax00.set_ylabel(ylabel, fontsize=12)
ax10.set_ylabel(ylabel, fontsize=12)
ax10.set_xlabel(xlabel, fontsize=12)
ax11.set_xlabel(xlabel, fontsize=12)

# Set titles
ax00.set_title('Selected w/ correct z', fontsize=15)
ax01.set_title('All selected', fontsize=15)
ax10.set_title('Purity', fontsize=15)
ax11.set_title('Completeness', fontsize=15)

plt.savefig('output/puri-comp_magcut-' + str(mag_cut) + '.pdf', dpi=600,
    bbox_inches='tight')
plt.show()

print('N_target = {}'.format(N_target))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))

puricomp_norm = (purity ** 2 + completeness ** 2) ** 0.5
sns.heatmap(puricomp_norm, ax=ax, cmap=cmap)
print(np.argmax(puricomp_norm))

ax.set_yticks(yticks)
ax.set_yticklabels(ytick_labels)
ax.set_xticks(xticks)
ax.set_xticklabels(xtick_labels, rotation='vertical')

ax.invert_yaxis()

ax.set_ylabel(ylabel, fontsize=12)
ax.set_xlabel(xlabel, fontsize=12)

ax.set_title(r'$\sqrt{\mathrm{Purity}^2 + \mathrm{Completeness}^2}$', fontsize=20)

plt.show()

In [ ]:
i = 3
j = 4

# Create synthetic BBs at the right and left of the line

lya_L = np.zeros(N_sources)
lya_R = np.zeros(N_sources)
lya_R2 = np.zeros(N_sources)
lya_L_err = np.zeros(N_sources) * 99
lya_R_err = np.zeros(N_sources) * 99
lya_R2_err = np.zeros(N_sources) * 99

for src in range(N_sources):
    if lya_select_list[i][src] == -1:
        continue
    l = lya_select_list[i][src]
    if l > 1:
        if l > 6:
            lya_L[src] = np.average(
                pm_flx[l - 7 : l - 1, src],
                weights=pm_err[l - 7 : l - 1, src] ** -2
            )
            lya_L_err[src] = np.sum(pm_err[l - 7 : l - 1, src] ** -2) ** -0.5
        else:
            lya_L[src] = np.average(
                pm_flx[:l - 1, src],
                weights=pm_err[:l - 1, src] ** -2
            )
            lya_L_err[src] = np.sum(pm_err[l - 7 : l - 1, src] ** -2) ** -0.5

    lya_R[src] = np.average(
        pm_flx[l + 1 : l + 8, src],
        weights=pm_err[l + 1 : l + 8, src] ** -2
    )
    lya_R2[src] = np.mean(pm_flx[l + 12 : l + 12 + 7, src])

    lya_R_err[src] = np.sum(pm_err[l + 1 : l + 8, src] ** -2) ** -0.5
    lya_R2_err[src] = np.sum(pm_err[l + 12 : l + 12 + 7, src] ** -2) ** -0.5

line_len_Arr = np.array([len(l) for l in other_select_list[j]])
nice_lya = (
    (np.array(lya_select_list[i]) != -1) & (line_len_Arr <= 1)
    # & np.invert(lya_L - lya_R > 3 * (lya_L_err ** 2 + lya_R_err ** 2) ** 0.5)
    # & (lya_R / lya_R2 > 1.)
)
nice_z = np.abs(lya_z_nb[i] - zspec) < 0.12

z_nb_cut = (lya_z_nb[i] > z_cut_min) & (lya_z_nb[i] < z_cut_max)

selected = mag_cut & nice_lya & z_nb_cut
print(len(np.where(selected)[0]))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))

LR_ratio = (lya_R / lya_R2)
bins = np.linspace(-2, 4, 40)

# ax.hist(LR_ratio, bins=bins, histtype='step', label='All')
ax.hist(LR_ratio[nice_lya & ~nice_z], bins=bins, histtype='step', label='Bad z')
ax.hist(LR_ratio[nice_lya & nice_z], bins=bins, histtype='step', label='Nice z')

ax.axvline(1, ls='--', color='gray')

ax.legend(fontsize=15)

ax.set_xlabel('$f_\mathrm{R}$ / $f_\mathrm{RR}$', fontsize=15)
ax.set_ylabel('N', fontsize=15)
plt.show()

In [ ]:
# for k, src in enumerate(np.random.choice(np.where(selected)[0], 20)):
wh = (lya_z_nb[i] > 3) & (lya_z_nb[i] < 4) & (mag[-2] < mag_cut) & selected
np.random.seed(23)
for k, src in enumerate(np.random.choice(np.where(wh)[0], 20)):
    if k == 20: break
    fig = plt.figure(figsize=(10, 6))
    ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])
    print(f'z_NB = {lya_z_nb[i][src]}')
    print(f'z_spec = {zspec[src]}')

    # l = lya_select_list[i][src]
    # if l != -1:
    #     ax.axvline(w_central[l], label='Selected NB')

    # for count, lo in enumerate(other_select_list[j][src]):
    #     if count == 0:
    #         ax.axvline(w_central[lo], label='Other lines', c='green')
    #     else:
    #         ax.axvline(w_central[lo], c='green')
    
    # ax.scatter(w_central[8:20], cont_est_lya[8:20, src], s=100, marker='s')
    ax.axvline(w_lya * (1 + zspec[src]), ls='--', c='red', label=r'Lya $\lambda$')
    ax.legend(fontsize=13)
    plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))
ax.hist2d(zspec[selected], lya_z_nb[i][selected],
    bins=(np.linspace(0, 4.5, 30), np.linspace(2.9, 4, 20)))
ax.set_xlabel('z$_\mathrm{spec}$', fontsize=15)
ax.set_ylabel('Retrieved z', fontsize=15)
plt.show()

In [ ]:
N_sel = len(np.where(selected)[0])
lya_L = np.zeros(N_sel)
lya_R = np.zeros(N_sel)
lya_L_err = np.zeros(N_sel)
lya_R_err = np.zeros(N_sel)

for j, src in enumerate(np.where(selected)[0]):
    l = lya_select_list[i][src]
    if l > 1:
        if l > 6:
            lya_L[j] = np.average(
                pm_flx[l - 7 : l - 1, src],
                weights=pm_err[l - 7 : l - 1, src] ** -2
            )
            lya_L_err[j] = np.sum(pm_err[l - 7 : l - 1, src] ** -2) ** -0.5
        else:
            lya_L[j] = np.average(
                pm_flx[:l - 1, src],
                weights=pm_err[:l - 1, src] ** -2
            )
            lya_L_err[j] = np.sum(pm_err[l - 7 : l - 1, src] ** -2) ** -0.5

    lya_R[j] = np.average(
        pm_flx[l + 1 : l + 6, src],
        weights=pm_err[l + 1 : l + 6, src] ** -2
    )
    lya_R_err[j] = np.sum(pm_err[l + 1 : l + 6, src] ** -2) ** -0.5

fig, ax = plt.subplots(figsize=(10, 8))
ax.hist(
    (lya_L - lya_R) / (lya_R_err**2 + lya_L_err**2) ** 0.5,
    bins=np.linspace(-5, 10, 50), histtype='step', label='Selected')

###

N_sel = len(np.where(selected & nice_z)[0])
lya_L = np.zeros(N_sel)
lya_R = np.zeros(N_sel)
lya_L_err = np.zeros(N_sel)
lya_R_err = np.zeros(N_sel)

for j, src in enumerate(np.where(selected & nice_z)[0]):
    l = lya_select_list[i][src]
    if l > 1:
        if l > 6:
            lya_L[j] = np.average(
                pm_flx[l - 7 : l - 1, src],
                weights=pm_err[l - 7 : l - 1, src] ** -2
            )
            lya_L_err[j] = np.sum(pm_err[l - 7 : l - 1, src] ** -2) ** -0.5
        else:
            lya_L[j] = np.average(
                pm_flx[:l - 1, src],
                weights=pm_err[:l - 1, src] ** -2
            )
            lya_L_err[j] = np.sum(pm_err[l - 7 : l - 1, src] ** -2) ** -0.5

    lya_R[j] = np.average(
        pm_flx[l + 1 : l + 6, src],
        weights=pm_err[l + 1 : l + 6, src] ** -2
    )
    lya_R_err[j] = np.sum(pm_err[l + 1 : l + 6, src] ** -2) ** -0.5

ax.hist(
    (lya_L - lya_R) / (lya_R_err**2 + lya_L_err**2) ** 0.5,
    bins=np.linspace(-5, 10, 50), histtype='step', label='Selected w/right z')

ax.legend(fontsize=15)
plt.show()